# Common Architectures in Keras    Shallow logistic regression, custom RBF-ish layer, CNN, feature hierarchy, and pretrained feature extractor using `tf.keras`.

In [ ]:
    # !pip install tensorflow scikit-learn matplotlib    import tensorflow as tf    from tensorflow import keras    from tensorflow.keras import layers    import numpy as np    import matplotlib.pyplot as plt    from sklearn.datasets import make_moons    tf.keras.utils.set_random_seed(123)    

## Shallow Model (logistic regression)

In [ ]:
    X = np.random.randn(400, 2)    true_w = np.array([[1.5], [-0.7]])    logits = X @ true_w + 0.2    y = (1 / (1 + np.exp(-logits)) > 0.5).astype(np.int32).ravel()    shallow = keras.Sequential([layers.Input(shape=(2,)), layers.Dense(1, activation='sigmoid')])    shallow.compile(optimizer=keras.optimizers.SGD(learning_rate=0.5), loss='binary_crossentropy', metrics=['accuracy'])    shallow.fit(X, y, epochs=60, verbose=0)    acc = shallow.evaluate(X, y, verbose=0)[1]    print(f"Logistic regression accuracy: {acc:.3f}")    

## RBF-style hidden layer (custom Lambda)

In [ ]:
    # RBF features computed against trainable centers    centers = tf.Variable(tf.random.normal([8, 2]))    sigma = tf.constant(1.0)    inputs = keras.Input(shape=(2,))    diff = tf.expand_dims(inputs, 1) - tf.expand_dims(centers, 0)    dist_sq = tf.reduce_sum(tf.square(diff), axis=2)    rbf_feat = tf.exp(-dist_sq / (2.0 * sigma ** 2))    rbf_out = layers.Dense(1, activation='sigmoid')(rbf_feat)    rbf_model = keras.Model(inputs, rbf_out)    rbf_model.compile(optimizer=keras.optimizers.Adam(0.05), loss='binary_crossentropy', metrics=['accuracy'])    rbf_model.fit(X, y, epochs=120, verbose=0)    rbf_acc = rbf_model.evaluate(X, y, verbose=0)[1]    print(f"RBF-like model accuracy: {rbf_acc:.3f}")    

## CNN on MNIST (small sample)

In [ ]:
    (x_train, y_train), _ = keras.datasets.mnist.load_data()    x_train = x_train[:500].astype('float32') / 255.0  # keep tiny subset for speed    y_train = y_train[:500]    x_train = np.expand_dims(x_train, -1)    cnn = keras.Sequential([        layers.Input(shape=(28, 28, 1)),        layers.Conv2D(8, 3, padding='same', activation='relu'),        layers.MaxPooling2D(),        layers.Conv2D(16, 3, padding='same', activation='relu'),        layers.MaxPooling2D(),        layers.Flatten(),        layers.Dense(32, activation='relu'),        layers.Dense(10, activation='softmax')    ])    cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])    cnn.fit(x_train, y_train, epochs=2, batch_size=64, verbose=1)    

## Hierarchical Feature Shapes

In [ ]:
    sample = x_train[0:1]    layer_outputs = [layer.output for layer in cnn.layers if isinstance(layer, layers.Conv2D)]    activation_model = keras.Model(inputs=cnn.input, outputs=layer_outputs)    feats = activation_model(sample)    for i, feat in enumerate(feats, start=1):        print(f"Conv layer {i} feature map shape: {feat.shape}")    

## Pretrained Feature Extractor (MobileNetV2)

In [ ]:
    from tensorflow.keras.applications import MobileNetV2    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input    base = MobileNetV2(weights=None, include_top=False, pooling='avg')  # set weights='imagenet' to download    base.trainable = False    dummy = np.random.rand(1, 224, 224, 3).astype('float32')    dummy_pp = preprocess_input(dummy.copy())    features = base(dummy_pp)    print("Feature vector shape:", features.shape)    